In this notebook we want to try an experiment with time prediction of the Sun. A model that contains encoder/decoder and LSTM is here defined but still unclear how to pass the SDO dataset (encoding decoding has to be applied by image)

In [1]:
import logging
import sys
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
from matplotlib.pyplot import imshow
import operator
from functools import reduce

import torch
from torch import nn, optim
from sdo.sdo_dataset import SDO_Dataset
from torch.utils.data import DataLoader
from torch.nn import functional as F

from sdo.models.encoder_decoder import AutoEncoder

%matplotlib inline

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, latent_dim=16):
        super(LSTMModel, self).__init__()
        # TODO: Choose a better hidden_size.
        # TODO: Choose more stacked num_layers
        self.lstm = nn.LSTM(input_size=latent_dim, hidden_size=50, num_layers=1, batch_first=True)
        self.fc1 = nn.Linear(in_features=50, out_features=latent_dim)
        
    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1]  # Keep only the output of the last iteration.
        x = self.fc1(x)
        x = nn.ReLU()(x)
        return x

In [7]:
#just a way to get nice logging messages from the sdo package
logformat = "[%(asctime)s] %(levelname)s:%(name)s:%(message)s"
logging.basicConfig(level=logging.DEBUG, stream=sys.stdout, format=logformat, datefmt="%Y-%m-%d %H:%M:%S")

In [8]:
# let's start simple by considering one single channel
# single channel has a bug in the data loader (one xextra dim is added)
subsample = 1
original_ratio = 512
img_shape = int(original_ratio/subsample)
instr = ['AIA', 'AIA']
channels = ['0171', '0094']

In [9]:
#some cuda initialization
torch.backends.cudnn.enabled = True
cuda_device = 3
if not torch.cuda.is_available():
    raise RuntimeError("CUDA not available! Unable to continue")
device = torch.device("cuda:{}".format(cuda_device))
print("Using device {} for training, current device: {}, total devices: {}".format(
device, torch.cuda.current_device(), torch.cuda.device_count()))

Using device cuda:3 for training, current device: 0, total devices: 6


In [10]:
%%time
train_data = SDO_Dataset(device=device, instr=instr, channels=channels, yr_range=[2018,2018], 
                         mnt_step=1, day_step=1, h_step=1, min_step=12, subsample=subsample, 
                         test_ratio=0.3, normalization=0, scaling=True)

[2019-07-23 12:55:56] INFO:sdo.sdo_dataset:Loading SDOML from "/gpfs/gpfs_gl4_16mb/b9p111/fdl_sw/SDOML"
[2019-07-23 12:55:56] INFO:sdo.sdo_dataset:Training on months "[1 2 3 4 5 6 7]"
[2019-07-23 12:55:56] DEBUG:sdo.sdo_dataset:Timestamps requested values: 
[2019-07-23 12:55:56] DEBUG:sdo.sdo_dataset:Years: 2018
[2019-07-23 12:55:56] DEBUG:sdo.sdo_dataset:Months: 1,2,3,4,5,6,7
[2019-07-23 12:55:56] DEBUG:sdo.sdo_dataset:Days: 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
[2019-07-23 12:55:56] DEBUG:sdo.sdo_dataset:Hours: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
[2019-07-23 12:55:56] DEBUG:sdo.sdo_dataset:Minutes: 0,12,24,36,48
[2019-07-23 12:55:56] INFO:sdo.sdo_dataset:Max number of timestamps: 26040
[2019-07-23 12:55:59] INFO:sdo.sdo_dataset:Timestamps found in the inventory: 24515 (0.94)
[2019-07-23 12:56:03] INFO:sdo.sdo_dataset:N timestamps discarded because channel is missing = 18 (0.00073)
[2019-07-23 12:56:03] INFO:sdo.s

In [11]:
"""
In this module we want to define an encoder/decoder RNN architecture
"""
def prod(iterable):
    return reduce(operator.mul, iterable, 1)


class EncoderDecoderRNN(nn.Module):
    """
    This is a RNN that output the next image of a sequence of images.
    """
    def __init__(self, input_shape=[1, 512, 512], hidden_dim_fc=512, hidden_dim_lstm=50):
        super(EncoderDecoderRNN, self).__init__()
        num_channels = input_shape[0]
        self.conv1 = nn.Conv2d(in_channels=num_channels, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3)
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3)
        
        self.dconv5 = nn.ConvTranspose2d(in_channels=64, out_channels=128, kernel_size=3)
        self.dconv4 = nn.ConvTranspose2d(in_channels=128, out_channels=128, kernel_size=3)
        self.dconv3 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3)
        self.dconv2 = nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=3)
        self.dconv1 = nn.ConvTranspose2d(in_channels=32, out_channels=num_channels, kernel_size=3)

        self.pool = nn.MaxPool2d(2, 2, return_indices=True)
        self.unpool = nn.MaxUnpool2d(2, 2)

        sample_encoder_input = torch.zeros(input_shape)
        sample_encoder_output = self._encoder(sample_encoder_input.unsqueeze(0))[0]
        sample_encoder_output_dim = sample_encoder_output.nelement()

        # TODO: Choose a better hidden_size.
        # TODO: Choose more stacked num_layers.
        self.lstm = nn.LSTM(input_size=sample_encoder_output_dim,
                            hidden_size=hidden_dim_lstm, num_layers=1, batch_first=True)
        self.lstm_fc = nn.Linear(in_features=hidden_dim_lstm,
                                 out_features=sample_encoder_output_dim)
        
        self.lin1 = nn.Linear(sample_encoder_output_dim, hidden_dim_fc)
        self.lin2 = nn.Linear(hidden_dim_fc, sample_encoder_output_dim)

        print('Autoencoder architecture:')
        print('Input shape: {}'.format(input_shape))
        print('Input dim  : {}'.format(prod(input_shape)))
        print('Encoded dim: {}'.format(sample_encoder_output_dim))
        print('Hidden dim FC: {}'.format(hidden_dim_fc))
        print('Learnable params: {}'.format(sum([p.numel() for p in self.parameters()])))

    def _encoder(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x, indices1 = self.pool(x)
        x = self.conv3(x)
        x, indices2 = self.pool(x)
        x = self.conv4(x)
        x, indices3 = self.pool(x)
        x = self.conv5(x)
        x, indices4 = self.pool(x)
        x = F.relu(x)
        return x, indices1, indices2, indices3, indices4

    def _decoder(self, x, indices1, indices2, indices3, indices4):
        x = self.unpool(x, indices4)
        x = self.dconv5(x)
        x = F.relu(x)
        x = self.unpool(x, indices3)
        x = self.dconv4(x)
        x = F.relu(x)
        x = self.unpool(x, indices2)
        x = self.dconv3(x)
        x = F.relu(x)
        x = self.unpool(x, indices1)
        x = self.dconv2(x)
        x = F.relu(x)
        x = self.dconv1(x)
        x = torch.relu(x)
        return x

    def forward(self, x):
        #import pdb
        #pdb.set_trace()
        
        batch_size = x.shape[0]
        x, indices1, indices2, indices3, indices4 = self._encoder(x)
        Shap = x.shape
        x = x.view(batch_size, -1)
        x = self.lin1(x)
        x = self.lin2(x)
        x = F.relu(x)
        x = x.reshape(Shap)
        
        x, _ = self.lstm(x)
        x = x[:, -1]  # Keep only the output of the last iteration.
        x = self.fc1(x)
        x = nn.ReLU()(x)
        
        x = self._decoder(x, indices1, indices2, indices3, indices4)
        return x

In [12]:
#let's define a RNN model
model = EncoderDecoderRNN()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_function = nn.MSELoss()

Autoencoder architecture:
Input shape: [1, 512, 512]
Input dim  : 262144
Encoded dim: 57600
Hidden dim FC: 512
Learnable params: 74136545


In [13]:
seq_len = 3 # Number of time steps for each sequence to learn over
batch_size = 1 # Number of independent sequences

In [14]:
# we are loading 2 channels, this is way we discard the first
start = 0
end = seq_len + 1
x = torch.tensor([[train_data[idx][0].cpu().numpy() for idx in range(start, end)]])
# Slice off final end step of sequence as the ground truth.
gt_final_step = x.clone()[:, -1]
x = x[:, 0:seq_len]
x = x.cuda()

In [ ]:
optimizer.zero_grad()
out = model(x)
loss = loss_function(out, gt_final_step)
loss.backward()
optimizer.step()

In [70]:
# is the shuffling in the data loader shuffling the order of the batches or inside the batch
train_data_loader = DataLoader(train_data, batch_size=1, shuffle=False)

In [ ]:
# training loop
model.cuda(cuda_device)
len_data = train_data.__len__()
log_interval = 10
# for now I am training on a single year
n_epochs = 4
train_loss = []
for epoch in range(n_epochs):
    for batch_index, batch in enumerate(train_data_loader):
        data = batch.to(cuda_device)
        optimizer.zero_grad()
        recon_batch = model(data)
        loss = loss_function(recon_batch, data)
        train_loss.append(float(loss))
        loss.backward()
        optimizer.step()
        if batch_index % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_index * len(data), len_data, 
                100.*(batch_index* len(data)) / len_data, 
                loss.item() / len(data)))